範例: 可供查詢的幣種 From https://www.binance.com/zh-TC
僅列舉幾種供測試用

幣種: BTC BNB ETH AAVE AXS DOGE SHIB ETC SOL AVAX XRP LINK MATIC TRX LUNA CAKE BAKE SAND BNX LPT COS TRU AR RNDR WLD AEVO C98 ID 
非USDT本位: WBTC/BTC ETH/BTC FLOKI/TRY SOL/BTC

NOTE: USDT 1:1 USD 
NOTE2: 下面有三套程式碼

必要的庫 (libs)

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

Binance 查詢價格與市值排名 (請勿將瀏覽器視窗向下還原)

In [5]:
"""
這裡是查詢價格與市值排名
正式版本
市場主導指數 是指該幣相對整個市場的份額

"""
print("格式: XXX/XXX , 範例: BTC/USDT ETH/BTC , 無輸入本位則預設USDT , 範例: BNB eth bTC(大小寫都可，中間空格)")
search = input("請問你要找哪些虛擬貨幣交易對?")
print("")
pairs = search.split() # 分割好幾個交易對

print("瀏覽器開啟中...")

driver = webdriver.Chrome()
driver.maximize_window() # 最大化瀏覽器視窗(因向下還原視窗組件會改變)
driver.get('https://www.binance.com/zh-TC/trade/')
driver.implicitly_wait(10)

# 找到包含所有選項的列表元素
list_elements = driver.find_element(By.CLASS_NAME, "css-lealug")

# 在列表下找到包含 "資訊" 文字的元素
information_element = list_elements.find_element(By.XPATH, "//div[contains(text(), '資訊')]")

# 點擊 "資訊" 元素
information_element.click()

# 創建一個空字典來存儲每個交易對的排名和市值
all_rank_volumn = {}

try:
    print("查找交易對中...")
    for pair in pairs:
        if '/' not in pair:
            pair += "/USDT"
        pair = pair.upper()
        print("")

        # 輸入交易對並點擊搜索按鈕
        search_input = driver.find_element(By.CSS_SELECTOR, "[class='trade-widget-search css-1fnphnh']")
        search_input.click()
        search_input.send_keys(Keys.CONTROL + "a")  # 選中輸入框中的文本
        search_input.send_keys(Keys.DELETE)         # 刪除選中的文本
        search_input.send_keys(pair)

        time.sleep(0.15) #暫停0.1秒等網頁元素跑好
        try:
            # 點擊交易對
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".item-symbol"))
            )
            element.click()

            # 獲取交易對名稱
            name_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "css-1qkv3vk"))
            )
            name = name_element.text

            # 獲取現價
            price_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "showPrice"))
            )
            price = price_element.text

            # 獲取24小時漲跌百分比
            updown_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "tickerPriceText"))
            )
            updown = updown_element.text.split()
            updownpercent = updown[1]
            
            # 進入資訊表
            info_element = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.coinInfo-item.css-3j2kqe div.title.css-1lywexu+div.content.css-vurnku"))
            )

            rank = info_element[0].text
            volumn = info_element[1].text
            Dominance_Index = info_element[2].text
            
            if "%" not in Dominance_Index: #有些特殊貨幣沒有市場主導指數
                Dominance_Index = "N/A" 

            print("交易對名稱:", name)
            print(f"現價({pair.split('/')[1]}):", price)
            print("24h漲跌:", updownpercent)
            print("目前市值:", volumn)
            print("")

            # 將交易對及其對應的排名和市值添加到字典中
            all_rank_volumn[name] = (rank, volumn, Dominance_Index)

        except TimeoutException:
            print("超時錯誤: 找不到交易對")
        except NoSuchElementException:
            print("元素未找到: 找不到交易對")

finally:
    print("查找完畢，關閉瀏覽器，統計進行中...")
    print("")
    driver.quit()  # 最後關閉瀏覽器

    # 排序字典，按排名排序
    sorted_all_rank_volumn = sorted(all_rank_volumn.items(), key=lambda x: int(x[1][0].split('：')[1]))

    # 打印排名結果
    print("交易對市值排名(USD):")
    for idx, (Stats_pair, (Stats_rank, Stats_volumn , Stats_Dominance_Index)) in enumerate(sorted_all_rank_volumn, start=1):
        print(f"{idx}. {Stats_pair}: 市值{Stats_volumn} , 市場主導指數:{Stats_Dominance_Index}")

格式: XXX/XXX , 範例: BTC/USDT ETH/BTC , 無輸入本位則預設USDT , 範例: BNB eth bTC(大小寫都可，中間空格)
請問你要找哪些虛擬貨幣交易對?BTC BNB ETH AAVE AXS DOGE SHIB ETC SOL AVAX XRP WBTC/BTC ETH/BTC FLOKI/TRY SOL/BTC

瀏覽器開啟中...
查找交易對中...

交易對名稱: BTC/USDT
現價(USDT): 65,877.99
24h漲跌: +1.22%
目前市值: $1,299,826.54M


交易對名稱: BNB/USDT
現價(USDT): 600.2
24h漲跌: +3.82%
目前市值: $88,752.68M


交易對名稱: ETH/USDT
現價(USDT): 3,200.54
24h漲跌: +1.31%
目前市值: $385,096.32M


交易對名稱: AAVE/USDT
現價(USDT): 95.52
24h漲跌: +4.03%
目前市值: $1,417.60M


交易對名稱: AXS/USDT
現價(USDT): 7.706
24h漲跌: +0.25%
目前市值: $1,107.80M


交易對名稱: DOGE/USDT
現價(USDT): 0.15948
24h漲跌: -1.11%
目前市值: $23,047.12M


交易對名稱: SHIB/USDT
現價(USDT): 0.00002699
24h漲跌: -0.22%
目前市值: $16,001.58M


交易對名稱: ETC/USDT
現價(USDT): 28.04
24h漲跌: +0.97%
目前市值: $4,117.50M


交易對名稱: SOL/USDT
現價(USDT): 153.03
24h漲跌: +1.63%
目前市值: $68,781.18M


交易對名稱: AVAX/USDT
現價(USDT): 39.49
24h漲跌: +5.62%
目前市值: $14,937.44M


交易對名稱: XRP/USDT
現價(USDT): 0.5324
24h漲跌: +0.55%
目前市值: $29,382.76M


交易對名稱: WBTC/BTC
現價(BTC): 1.0009
24h漲跌: -0.04%
目前市值: $

以下是交易連續按鍵自動化部分，需要登入才能進行，開啟後要到網站裡面進行登錄 #以下是市價買賣

In [121]:
'''

交易連續按鍵自動化，需要登入才能進行，開啟後要到網站裡面進行登錄 預設成市價(現價)

'''
print("格式: XXX/XXX , 範例: BTC/USDT")
pair = input("請問您要買賣的虛擬貨幣交易對?")
print("")

print("瀏覽器開啟中...")

try:
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get('https://www.binance.com/zh-TC/trade/')
    driver.implicitly_wait(10)

    search_input = driver.find_element(By.CSS_SELECTOR, "[class='trade-widget-search css-1fnphnh']")
    search_input.click()
    search_input.send_keys(Keys.CONTROL + "a")  # 選中輸入框中的文本
    search_input.send_keys(Keys.DELETE)         # 刪除選中的文本
    search_input.send_keys(pair)
    time.sleep(0.15) #暫停0.15秒等網頁元素跑好
       
    element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, ".item-symbol"))
    )
    element.click() # 點擊交易對

    print("請前往進行登錄")
    ask = str(input("您要 買/賣?"))

    num_trades = int(input("您希望執行多少次連續交易？"))

    if ask == "買":
        quantity = int(input("您每次要買多少成交額(以交易對後面那個為計價)?")) 

        buyq = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.ID, "FormRow-BUY-total"))
        )
        buyq.click()
        buyq.send_keys(quantity)

        buyButton = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "[class='buySellButton css-1qka0vb']"))
        )
        for times in range(num_trades):
            try:
                buyButton.click()
            except Exception as e:
                print(f"在第 {times+1} 次交易時出現錯誤：{str(e)}")

    else:
        quantity = int(input("您要每次要賣多少成交額(以交易對後面那個為計價)?"))

        sellq = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.ID, "FormRow-SELL-total"))
        )
        sellq.click()
        sellq.send_keys(quantity)

        sellButton = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "[class='buySellButton css-ty50m1']"))
        )
        for times in range(num_trades):
            try:
                sellButton.click()
            except Exception as e:
                print(f"在第 {times+1} 次交易時出現錯誤：{str(e)}")

except Exception as e:
    print(f"發生錯誤：{str(e)}")

finally:
    if 'driver' in locals():
        driver.quit()


格式: XXX/XXX , 範例: BTC/USDT
請問您要買賣的虛擬貨幣交易對?bnb/usdt

瀏覽器開啟中...
請前往進行登錄
您要 買/賣?買
您要買多少成交額(以交易對後面那個為計價)?10000


以下為運作細節展示版本

In [122]:
"""
這裡是查詢價格與市值排名
會Print每個操作時細節的版本
市場主導指數 是指該幣相對整個市場的份額

"""

from selenium.common.exceptions import TimeoutException, NoSuchElementException

print("格式: XXX/XXX , 範例: BTC/USDT ETH/BTC , 無輸入本位則預設USDT , 範例: BNB eth bTC(大小寫都可，中間空格)")
search = input("請問你要找哪些虛擬貨幣交易對?")
print("")
pairs = search.split() # 分割好幾個交易對

print("瀏覽器開啟中...")

driver = webdriver.Chrome()
driver.maximize_window() # 最大化瀏覽器視窗(因向下還原視窗組件會改變)
driver.get('https://www.binance.com/zh-TC/trade/')
driver.implicitly_wait(10)

# 找到包含所有選項的列表元素
# 找到包含所有選項的列表元素
list_elements = driver.find_element(By.CLASS_NAME, "css-lealug")

# 在列表下找到包含 "資訊" 文字的元素
information_element = list_elements.find_element(By.XPATH, "//div[contains(text(), '資訊')]")

# 點擊 "資訊" 元素
information_element.click()

# 創建一個空字典來存儲每個交易對的排名和市值
all_rank_volumn = {}

try:
    print("查找交易對中...")
    for pair in pairs:
        if '/' not in pair:
            pair += "/USDT"
        pair = pair.upper()
        print("")

        # 輸入交易對並點擊搜索按鈕
        search_input = driver.find_element(By.CSS_SELECTOR, "[class='trade-widget-search css-1fnphnh']")
        search_input.click()
        search_input.send_keys(Keys.CONTROL + "a")  # 選中輸入框中的文本
        search_input.send_keys(Keys.DELETE)         # 刪除選中的文本
        search_input.send_keys(pair)

        time.sleep(0.15) #暫停0.15秒等網頁元素跑好
        try:
            # 點擊交易對
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".item-symbol"))
            )
            element.click()

            # 獲取交易對名稱
            name_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "css-1qkv3vk"))
            )
            name = name_element.text

            # 獲取現價
            price_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "showPrice"))
            )
            price = price_element.text

            # 獲取24小時漲跌百分比
            updown_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "tickerPriceText"))
            )
            updown = updown_element.text.split()
            updownpercent = updown[1]
            
            # 進入資訊表
            info_element = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.coinInfo-item.css-3j2kqe div.title.css-1lywexu+div.content.css-vurnku"))
            )

            rank = info_element[0].text
            volumn = info_element[1].text
            Dominance_Index = info_element[2].text
            
            if "%" not in Dominance_Index: #有些特殊貨幣沒有市場主導指數
                Dominance_Index = "N/A" 

            print("交易對名稱:", name)
            print(f"現價({pair.split('/')[1]}):", price)
            print("24h漲跌:", updownpercent)
            print("目前市值:", volumn)
            print("")

            # 將交易對及其對應的排名和市值添加到字典中
            all_rank_volumn[name] = (rank, volumn, Dominance_Index)
            print("目前字典: ", all_rank_volumn)

        except TimeoutException:
            print("超時錯誤: 找不到交易對")
        except NoSuchElementException:
            print("元素未找到: 找不到交易對")

finally:
    print("")
    print("查找完畢，關閉瀏覽器，統計進行中...")
    print("")
    driver.quit()  # 最後關閉瀏覽器

    print("轉為字典視圖: ", all_rank_volumn.items())
    # 排序字典，按排名排序
    sorted_all_rank_volumn = sorted(all_rank_volumn.items(), key=lambda x: int(x[1][0].split('：')[1]))
    print("")
    print("轉為排好的列表: ", sorted_all_rank_volumn)
    print("")

    # 打印排名結果
    print("交易對市值排名(USD):")
    for idx, (Stats_pair, (Stats_rank, Stats_volumn , Stats_Dominance_Index)) in enumerate(sorted_all_rank_volumn, start=1):
        print(f"{idx}. {Stats_pair}: 市值{Stats_volumn} , 市場主導指數:{Stats_Dominance_Index}")

格式: XXX/XXX , 範例: BTC/USDT ETH/BTC , 無輸入本位則預設USDT , 範例: BNB eth bTC(大小寫都可，中間空格)
請問你要找哪些虛擬貨幣交易對?bnb usdc eth btc bnx shib doge

瀏覽器開啟中...
查找交易對中...

交易對名稱: BNB/USDT
現價(USDT): 580.5
24h漲跌: +4.28%
目前市值: $86,822.21M

目前字典:  {'BNB/USDT': ('編號：4', '$86,822.21M', '3.62%')}

交易對名稱: USDC/USDT
現價(USDT): 0.9996
24h漲跌: -0.01%
目前市值: $33,953.90M

目前字典:  {'BNB/USDT': ('編號：4', '$86,822.21M', '3.62%'), 'USDC/USDT': ('編號：6', '$33,953.90M', '1.42%')}

交易對名稱: ETH/USDT
現價(USDT): 3,168.89
24h漲跌: +4.07%
目前市值: $380,299.93M

目前字典:  {'BNB/USDT': ('編號：4', '$86,822.21M', '3.62%'), 'USDC/USDT': ('編號：6', '$33,953.90M', '1.42%'), 'ETH/USDT': ('編號：2', '$380,299.93M', '15.9%')}

交易對名稱: BTC/USDT
現價(USDT): 64,852.38
24h漲跌: +1.87%
目前市值: $1,278,837.77M

目前字典:  {'BNB/USDT': ('編號：4', '$86,822.21M', '3.62%'), 'USDC/USDT': ('編號：6', '$33,953.90M', '1.42%'), 'ETH/USDT': ('編號：2', '$380,299.93M', '15.9%'), 'BTC/USDT': ('編號：1', '$1,278,837.77M', '53.42%')}

交易對名稱: BNX/USDT
現價(USDT): 0.7068
24h漲跌: +1.06%
目前市值: $254.27M

目前字典:  {'BN